# Microbiome Analysis

All the analysis here is done using the characteristic signature of each node as a featurization of each bacteria's status in the bacterial community.


In [12]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sb
import networkx as nx
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('../')

## Computing Distances between graphs

In [18]:
### Load/ or Compute the associated characteristic signature
chi=np.load("/Users/cdonnat/Dropbox/GSP/tests_real/microbiome_analysis/cached_quantities/chi_corr_nx.npy")
chi=np.ndarray.flatten(chi)[0] #since it's a little long, we just use the cached files


### Compute Distances between samples from the characteristic functions
D=pd.DataFrame(np.zeros((162,162)),index=sample_names)
D.columns=sample_names
for i in range(1,162):
    print i
    for j in range(i):
        print j
        c1=sample_names[i]
        c2=sample_names[j]
        D.loc[c1,c2]=chi_distance_aligned_graph([n for n in Graphs_sel[c1].index],[n for n in Graphs_sel[c2].index],chi[c1],chi[c2],nb_t=len(taus))
D=D+D.T
np.save("cached_quantities/D_harm.npy",D)


NameError: name 'sample_names' is not defined

## Analysis of the Graphs dynamics

We use the distance matrix that we just computed to visualize the clustering of the sampel

In [ ]:
from sklearn.manifold import TSNE
from sklearn.manifold import MDS
#model = TSNE(n_components=2, random_state=0,metric='precomputed')
model=sklearn.manifold.MDS(n_components=2, metric=True, n_init=4, max_iter=300, verbose=0, eps=0.001, n_jobs=1, random_state=None, dissimilarity='precomputed')
np.set_printoptions(suppress=True)

#chi_new=pca.fit_transform(chi[k])
chi_new=model.fit_transform(D) 
colors=[]
for e in sample_names:
    if e[0]=="E":
        colors.append("royalblue")
    elif e[0]=="D":
        colors.append("palevioletred")
    elif e[0]=="F":
        colors.append("darkturquoise")
        

plot=True
savefig=True
annotate=[True,"color"]
colors_state={e:colors_evol_E[e] for e in colors_evol_E.keys()}
colors_state.update(colors_evol_D)
colors_state.update(colors_evol_F)
if plot==True:
    fig,ax=plt.subplots(figsize=(10,12))
    sb.set_style("white")
    sb.set(style='ticks')
    sb.set_style({"xtick.direction":"in", "ytick.direction":"in"})
    sb.set_context("paper", font_scale=1.0)
    cmap = plt.get_cmap('hot')
    plt.scatter(chi_new[:,0],chi_new[:,1],cmap=cmap,s=300,c=[colors_state[e] for e in D.index])
    for label, x, y in zip(sample_names, chi_new[:, 0], chi_new[:, 1]):
                #print label,x,y
                plt.annotate(label,xy=(x, y), xytext=(-1, 0), textcoords='offset points')
    
    #plt.title("PCA projection of the wavelet distribution")
    plt.tight_layout()
    plt.savefig("plots_microbiome_analysis/MDS_ABT_colored_state.pdf")

## Evolution of the "Sample" graphs

The goal here is to capture the dynamics of the sample graphs as a whole.


In [ ]:
index_E=[e for e in sample_names if e[0]=="E"]
index_F=[e for e in sample_names  if e[0]=="F"]
index_D=[e for e in sample_names  if e[0]=="D"]

states=[u for u in np.unique(sample.loc[:,"condition"])]
color=["maroon","indianred", "orange","gold","lightgreen","mediumseagreen", "royalblue"]
evol_distance_E=np.zeros(len(index_E)-1)
colors_evol_E={e:color[states.index(sample.loc[e,"condition"])] for e in index_E }
ordered_color_E=[]
for e in range(1,len(index_E)):
    e1="E"+str(e)
    e2="E"+str(e+1)
    ordered_color_E.append(colors_evol_E[e1])
    evol_distance_E[e-1]= chi_distance_aligned_graph([n for n in Graphs_sel[e1].index],[n for n in Graphs_sel[e2].index],chi[e1],chi[e2],nb_t=len(taus))



fig,ax=plt.subplots(figsize=(10,12))
sb.set(style='ticks')
sb.set_context("paper", font_scale=1.5)
for col in color:
    ind=[u for u in range(len(ordered_color_E)) if ordered_color_E[u]==col]
    if ind[-1]+1<(len(ordered_color_E)-1):
        ind+=[ind[-1]+1]  
    plt.plot(ind,[evol_distance_E[i] for i in ind],c=col,label=states[color.index(col)])
plt.legend(loc="upper right")
plt.savefig("plots_microbiome_analysis/harm_evol_E.pdf")


evol_distance_F=np.zeros(len(index_F)-1)
colors_evol_F={e:color[states.index(sample.loc[e,"condition"])] for e in index_F }
ordered_color_F=[]
for e in range(1,len(index_F)):
    e1="F"+str(e)
    e2="F"+str(e+1)
    ordered_color_F.append(colors_evol_F[e1])
    evol_distance_F[e-1]= chi_distance_aligned_graph([n for n in Graphs_sel[e1].index],[n for n in Graphs_sel[e2].index],chi[e1],chi[e2],nb_t=len(taus))



fig,ax=plt.subplots(figsize=(10,12))
sb.set(style='ticks')
sb.set_context("paper", font_scale=1.5)
for col in color:
    ind=[u for u in range(len(ordered_color_F)) if ordered_color_F[u]==col]
    if ind[-1]+1<(len(ordered_color_F)-1):
        ind+=[ind[-1]+1]  
    plt.plot(ind,[evol_distance_F[i] for i in ind],c=col,label=states[color.index(col)])
plt.legend(loc="upper right")
plt.savefig("plots_microbiome_analysis/harm_evol_F.pdf")



evol_distance_D=np.zeros(len(index_D)-1)
colors_evol_D={e:color[states.index(sample.loc[e,"condition"])] for e in index_D }
ordered_color_D=[]
for e in range(1,len(index_D)):
    e1="D"+str(e)
    e2="D"+str(e+1)
    ordered_color_D.append(colors_evol_D[e1])
    evol_distance_D[e-1]= chi_distance_aligned_graph([n for n in Graphs_sel[e1].index],[n for n in Graphs_sel[e2].index],chi[e1],chi[e2],nb_t=len(taus))



fig,ax=plt.subplots(figsize=(10,12))
sb.set(style='ticks')
sb.set_context("paper", font_scale=1.5)
for col in color:
    ind=[u for u in range(len(ordered_color_D)) if ordered_color_D[u]==col]
    if ind[-1]+1<(len(ordered_color_D)-1):
        ind+=[ind[-1]+1]  
    plt.plot(ind,[evol_distance_D[i] for i in ind],c=col,label=states[color.index(col)])
plt.legend(loc="upper right")
plt.savefig("plots_microbiome_analysis/harm_evol_D.pdf")


## Understanding the evolution on a "per-bacteria" level

The goal here is to map the evolution of a bacteria's status within the graph. We select a few bacteria (of interest?), and see how their status (as reflected by the diffusion distribution centered around each and every one of them) evolves over time.

In [ ]:
bacs=["Unc06grq","Unc09fy6","Unc06d5x","PabDis18"]
d_taxa={bac:np.zeros(51) for bac in bacs}
node=0
mode=2
for bac in bacs:
    for i in range(1,51):
        print i
        e1="E"+str(i)
        e2="E"+str(i+1)
        if bac in Graphs_sel[e1].index:
            ind_list=[u for u in (Graphs_sel[e1].index)]
            ind1=ind_list.index(bac)
        else:
            ind1=np.nan
        if bac in Graphs_sel[e2].index:
            ind_list2=[u for u in (Graphs_sel[e2].index)]
            ind2=ind_list2.index(bac)
        else:
            ind2=np.nan
        if np.isnan(ind1) and np.isnan(ind2):
            d=0
        elif np.isnan(ind1):
            print "here"
            d=np.linalg.norm(heat_print[e2][mode].iloc[:,ind2])
        elif np.isnan(ind2):
            print "hhere"
            d=np.linalg.norm(heat_print[e1][mode].iloc[:,ind1])
        else:
            sig1=heat_print[e1][mode].iloc[:,ind1]
            sig2=heat_print[e2][mode].iloc[:,ind2]
            d=compute_auc(sig1,sig2,mode_diff="agg",normalize=True)
    
        d_taxa[bac][i]=d
d_taxa[bacs[1]][15]=0

it=0
fig,ax=plt.subplots()
color=["royalblue","indianred", "lightseagreen", "lemonchiffon","indianred","sierra"]
for bac in bacs:
    plt.plot(d_taxa[bac],c=color[it],label=bac)
    it+=1
plt.xlabel("Time")
plt.ylabel("Distances between consecutive diffusion patterns")
plt.legend(loc="upper right")
plt.savefig("plots_microbiome_analysis/evol_node.pdf")